<a href="https://colab.research.google.com/github/srvmishra/Language-Models/blob/main/Finetuning_DistilBERT_Base_Uncased_for_MLM_on_IMDb_data_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

### Imports

In [2]:
import numpy as np
import math
import collections
import pandas as pd
import torch
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer
from transformers import default_data_collator

from huggingface_hub import notebook_login

### Load Model and Tokenizer

In [3]:
model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForMaskedLM.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

### Run on an Example

In [4]:
text = "This is a great [MASK]."

model_inputs = tokenizer(text, return_tensors='pt')
model_outputs = model(**model_inputs).logits
mask_index = torch.where(model_inputs['input_ids'] == tokenizer.mask_token_id)[1]
mask_token_output = model_outputs[0, mask_index, :]
top_5_tokens = torch.topk(mask_token_output, 5, dim=-1).indices[0].tolist()

for token_id in top_5_tokens:
  print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token_id]))}")

This is a great deal.
This is a great success.
This is a great adventure.
This is a great idea.
This is a great feat.


### Load and Process Dataset

In [5]:
movie_data = load_dataset('imdb')
movie_data

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
print(movie_data['train'].features)
print(movie_data.column_names)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}


In [7]:
train_samples = movie_data['train'].shuffle(42).select(range(5))
for sample in train_samples:
  print('Text: ', sample['text'])
  print('Label: ', sample['label'])
print('###############################')
unsupervised_samples = movie_data['unsupervised'].shuffle(42).select(range(5))
for sample in unsupervised_samples:
  print('Text: ', sample['text'])
  print('Label: ', sample['label'])
print('###############################')
test_samples = movie_data['test'].shuffle(42).select(range(5))
for sample in test_samples:
  print('Text: ', sample['text'])
  print('Label: ', sample['label'])
print('###############################')

Text:  There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
Label:  1
Text:  This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the mo

### Tokenize Data

In [8]:
def tokenize(example):
  tokenized_examples = tokenizer(example['text'])
  if tokenizer.is_fast:
    tokenized_examples['word_ids'] = [tokenized_examples.word_ids(i) for i in range(len(tokenized_examples['input_ids']))]
  return tokenized_examples

In [9]:
concatenated_tokenized_dataset = movie_data.map(tokenize, batched=True,
                                                remove_columns=movie_data['train'].column_names)
concatenated_tokenized_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

### Concatenate and Chunk Data

In [10]:
chunk_size = 128

def process(examples):
  # concatenate
  concatenated_samples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_samples[list(concatenated_samples.keys())[0]])
  # drop the last sequence that may be the smallest
  chunked_length = (total_length // chunk_size) * chunk_size
  # chunk each field
  results = {k: [t[i: i+chunk_size] for i in range(0, chunked_length, chunk_size)] for k, t in concatenated_samples.items()}
  # masked language modeling: input ids and labels are the same: take from results, not from concatenated_samples
  results['labels'] = results['input_ids'].copy()
  return results

In [11]:
chunked_dataset = concatenated_tokenized_dataset.map(process, batched=True)
chunked_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

### Mask Random Tokens

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [13]:
samples = [chunked_dataset['train'][i] for i in range(2)]
for sample in samples:
  _ = sample.pop('word_ids')

for chunk in data_collator(samples)['input_ids']:
  print(f"\n>>>'{tokenizer.decode(chunk)}'")


>>>'[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it whenac was first released in 1967 bal i also heard that at first it was seized [MASK] u. s. customs [MASK] it ever tried to enter [MASK] country, therefore being a fan [MASK] films considered " controversial [MASK] [MASK] really had to see this for myself. < br / > < [MASK] / > the plot is centered [MASK] a [MASK] swedish [MASK] student [MASK] lena who wants to learn everything she can about life. in particular she wants to [MASK] her [MASK] rebels to making [MASK] sort of documentary on [MASK] the average swede [MASK] [MASK] certain political issues such'

>>>'as the vietnam war and [MASK] issues in the united states. in between asking politicians and ordinary denizens of stockholm about their [MASK] on politics [MASK] she has sex with her drama teacher, classmates, [MASK] married [MASK]. < br / > < br / > [MASK] kills me about i am curious - yellow is that 40 years ago, thi

### Mask Whole Words

In [14]:
wwm_probability = 0.2

# go through this function once
def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [17]:
samples = [chunked_dataset['train'][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch['input_ids']:
  print(f"\n>>>'{tokenizer.decode(chunk)}'")


>>>'[CLS] i rented i [MASK] [MASK] - yellow from my video store because [MASK] all the controversy that surrounded it when it was first released in 1967. i [MASK] [MASK] that at first it was seized by u [MASK] s [MASK] customs if it ever tried to enter [MASK] [MASK], therefore being a fan of films considered [MASK] controversial [MASK] i [MASK] had [MASK] see this for myself. < [MASK] / [MASK] < br / > the plot is centered around a young [MASK] drama student named [MASK] who wants to learn everything she can about [MASK] [MASK] in particular [MASK] wants to focus [MASK] [MASK] [MASK] to making some sort of documentary [MASK] what the average swede thought [MASK] [MASK] political [MASK] such'

>>>'as the vietnam war and race issues in [MASK] united states. in between asking politicians [MASK] ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher [MASK] classmates [MASK] [MASK] married men [MASK] < br / > < br / > [MASK] kills me about i [MA

### Down Sample the Dataset

In [18]:
train_size = 10000
test_size = int(0.1 * train_size)

downsampled_dataset = chunked_dataset['train'].train_test_split(train_size=train_size, test_size=test_size, seed=42)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

### Finetune the Model with Data Collator for Language Modeling

In [19]:
notebook_login()

In [21]:
batch_size = 64
logging_steps = len(downsampled_dataset['train'])//batch_size
model_name = f'srvmishra832/IMDb_data_subset-MLM_with-inbuilt_collator-{model_ckpt}'

training_arguments = TrainingArguments(output_dir=model_name, log_level='error',
                                       evaluation_strategy='epoch', logging_steps=logging_steps,
                                       num_train_epochs=5, learning_rate=2e-5, weight_decay=0.01,
                                       per_device_train_batch_size=batch_size,
                                       per_device_eval_batch_size=batch_size,
                                       fp16=True, push_to_hub=True)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(model=model, args=training_arguments,
                  train_dataset=downsampled_dataset['train'],
                  eval_dataset=downsampled_dataset['test'],
                  data_collator=data_collator, tokenizer=tokenizer)

<ipython-input-22-609bdca99825>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_arguments,


In [24]:
results_before_train = trainer.evaluate()
print(f"Before training perplexity: {math.exp(results_before_train['eval_loss']):.2f}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: srvmishra832 (srvmishra832-indian-institute-of-science-bangalore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Before training perplexity: 21.94


In [25]:
trainer.train()
trainer.push_to_hub()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.682400,2.502150,0.001800
2,2.580000,2.441642,0.001800
3,2.511700,2.464087,0.001800
4,2.501300,2.379560,0.001800
5,2.483500,2.422045,0.001800


events.out.tfevents.1742927108.49429658639f.240.0:   0%|          | 0.00/8.49k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srvmishra832/IMDb_data_subset-MLM_with-inbuilt_collator-distilbert-base-uncased/commit/2fc7cbd7268b44c4009953a3fcc331ad268e08b3', commit_message='End of training', commit_description='', oid='2fc7cbd7268b44c4009953a3fcc331ad268e08b3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/srvmishra832/IMDb_data_subset-MLM_with-inbuilt_collator-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='srvmishra832/IMDb_data_subset-MLM_with-inbuilt_collator-distilbert-base-uncased'), pr_revision=None, pr_num=None)

In [26]:
results_after_train = trainer.evaluate()
print(f"After training perplexity: {math.exp(results_after_train['eval_loss']):.2f}")

After training perplexity: 10.89


### Finetune Model with Custom Collator

In [45]:
# define model again for starting from pretrained weights checkpoint
model = AutoModelForMaskedLM.from_pretrained(model_ckpt)

In [46]:
batch_size = 64
logging_steps = len(downsampled_dataset['train'])//batch_size
model_name = f'srvmishra832/IMDb_data_subset-MLM_with-custom_collator-{model_ckpt}'

training_arguments = TrainingArguments(output_dir=model_name, log_level='error',
                                       evaluation_strategy='epoch', logging_steps=logging_steps,
                                       num_train_epochs=5, learning_rate=2e-5, weight_decay=0.01,
                                       per_device_train_batch_size=batch_size,
                                       per_device_eval_batch_size=batch_size, disable_tqdm=False,
                                       fp16=True, push_to_hub=True, remove_unused_columns=False)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
trainer = Trainer(model=model, args=training_arguments,
                  train_dataset=downsampled_dataset['train'],
                  eval_dataset=downsampled_dataset['test'],
                  data_collator=whole_word_masking_data_collator, tokenizer=tokenizer)

<ipython-input-47-d48d5f0ebeaf>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_arguments,


In [48]:
results_before_train = trainer.evaluate()
print(f"Before training perplexity: {math.exp(results_before_train['eval_loss']):.2f}")

Before training perplexity: 62.48


In [49]:
trainer.train()
trainer.push_to_hub()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,3.557900,3.305812,0.004200
2,3.394500,3.273233,0.004200
3,3.348700,3.254203,0.004200
4,3.308800,3.223724,0.004200
5,3.296100,3.253809,0.004200


events.out.tfevents.1742927904.49429658639f.240.3:   0%|          | 0.00/8.48k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srvmishra832/IMDb_data_subset-MLM_with-custom_collator-distilbert-base-uncased/commit/37102cd4400f62a03703170c8c7947a85806d013', commit_message='End of training', commit_description='', oid='37102cd4400f62a03703170c8c7947a85806d013', pr_url=None, repo_url=RepoUrl('https://huggingface.co/srvmishra832/IMDb_data_subset-MLM_with-custom_collator-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='srvmishra832/IMDb_data_subset-MLM_with-custom_collator-distilbert-base-uncased'), pr_revision=None, pr_num=None)

In [50]:
results_after_train = trainer.evaluate()
print(f"After training perplexity: {math.exp(results_after_train['eval_loss']):.2f}")

After training perplexity: 25.87
